# Produção dos gráficos de Tweets vs. Indicadores

In [1]:
import pandas as pd
from datetime import datetime
import os

# Bases de dados

Os gráficos gerados nesse notebook utilizam os seguintes dados:

    - Novos casos por semana
    - Novos casos por semana(MM)
    - Número de Tweets
    - Número de Retweets
    - Número de Tweets + Retweets

<b>Observações:</b> O número de Tweets não contém os Retweets; Pode-se utilizar quaisquer indicadores presentes no arquivo `indicadores.csv`.

 ### Leitura e tratamento dos indicadores

In [4]:
indicadores = pd.read_csv('indicadores.csv')
indicadores = indicadores[indicadores['region'] == 'Brasil'] # Separando apenas os dados do Brasil (descartando as Regiões)
indicadores = indicadores.sort_values('week_last_day') # Ordenando pelo último dia da semana, em vez de semana epidemiológica
indicadores['week_last_day'] = indicadores['week_last_day'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y')) # Convertendo os dias de String para Datetime

indicadores = indicadores[['week_last_day', 'new_week_cases', 'new_week_cases_mm']] # Retirando demais indicadores, mantendo apenas o "new_week_cases"
indicadores.columns = ['date', 'nw_cases', 'nw_cases_mm'] # Renomeando as colunas
indicadores = indicadores.sort_values(by='date') # Ordenando por data
indicadores['date'] = pd.to_datetime(indicadores['date']) # Convertendo as datas para Timestamps do Pandas

indicadores = indicadores[indicadores['date'] >= pd.to_datetime('2020-05-09')] # Cortando primeiras semanas para parear com o "df"

### Lendo bases de dados

In [8]:
df = pd.read_csv('tweets_dates.csv')
rt_df = pd.read_csv('retweets_dates.csv')

Gerando um Dataframe com o número total de tweets

In [10]:
total_df = pd.DataFrame.copy(df)
total_df['count'] += rt_df['count']

# Plotagem de gráficos

Aqui, são possíveis várias combinações. Por isso as plotagens são extensas, mas todas semelhantes uma à outra.

# Tweets e Retweets

### Plotando gráfico da razão <i>Tweets</i> por semana / <i>Reweets</i> por semana

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(y=df['count']/rt_df['count'], x=df['date'],  name='<i>Tweets/Retweets</i> por semana'))

# Add figure title
fig.update_layout(
    title_text="Razão <i>Tweets</i>/<i>Retweets</i> por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="<i>Tweets</i>/Retweets por semana", range=[0,1])

fig.write_html('tweets_over_retweets.html')
fig.write_image('svg/tweets_over_retweets.svg', width=1280, height=720)

### Plotando gráfico <i>Tweets</i> por semana vs. <i>Reweets</i> por semana

In [116]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(y=df['count'], x=df['date'],  name='<i>Tweets</i> por semana'))
fig.add_trace(go.Scatter(y=rt_df['count'], x=rt_df['date'], name='<i>Retweets</i> por semana'))

# Add figure title
fig.update_layout(
    title_text="<i>Tweets</i> por semana vs. <i>Retweets</i> por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="<i>Tweets</i> por semana", secondary_y=True)

fig.write_html('tweets_vs_retweets.html')
fig.write_image('svg/tweets_vs_retweets.svg', width=1280, height=720)

### Plotando gráfico <i>Tweets</i> por semana vs. <i>Tweets + Reweets</i> por semana

In [117]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(y=df['count'], x=df['date'],  name='<i>Tweets</i> por semana'))
fig.add_trace(go.Scatter(y=rt_df['count'], x=rt_df['date'], name='<i>Retweets</i> por semana'))

# Add figure title
fig.update_layout(
    title_text="<i>Tweets</i> por semana vs. <i>Retweets</i> por semana"
)

# Set y-axes titles
fig.update_yaxes(title_text="<i>Tweets</i> por semana", secondary_y=True)

fig.write_html('tweets_vs_retweets.html')
fig.write_image('svg/tweets_vs_retweets.svg', width=1280, height=720)

# Novos casos por semana e Tweets

### Plotando gráfico <i>Tweets</i> por semana vs. Novos casos por semana

In [118]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=df['count'], x=df['date'], mode='lines', name='Tweets por semana'), secondary_y=True)
fig.add_trace(go.Scatter(y=indicadores['nw_cases'], x=indicadores['date'], mode='lines', name='Novos casos por semana'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="<i>Tweets</i> por semana vs. Novos casos por semana"
)

# Set y-axes titles
fig.update_yaxes(title_text="Novos casos por semana", secondary_y=False)
fig.update_yaxes(title_text="Tweets por semana", secondary_y=True)

fig.write_html('cases_vs_tweets.html')
fig.write_image('svg/cases_vs_tweets.svg', width=1280, height=720)

### Plotando gráfico <i>Tweets</i> por semana vs. Novos casos por semana (MM)

In [119]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=df['count'], x=df['date'], mode='lines', name='Tweets por semana'), secondary_y=True)
fig.add_trace(go.Scatter(y=indicadores['nw_cases_mm'], x=indicadores['date'], mode='lines', name='Novos casos por semana (MM)'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="<i>Tweets</i> por semana vs. Novos casos por semana (MM)"
)

# Set y-axes titles
fig.update_yaxes(title_text="Novos casos por semana (MM)", secondary_y=False)
fig.update_yaxes(title_text="Tweets por semana", secondary_y=True)

fig.write_html('cases_vs_tweets_mm.html')
fig.write_image('svg/cases_vs_tweets_mm.svg', width=1280, height=720)

# Novos casos por semana e Retweets

### Plotando gráfico <i>Retweets</i> por semana vs. Novos casos por semana

In [120]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=rt_df['count'], x=rt_df['date'], mode='lines', name='<i>Retweets</i> por semana'), secondary_y=True)
fig.add_trace(go.Scatter(y=indicadores['nw_cases'], x=indicadores['date'], mode='lines', name='Novos casos por semana'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="<i>Retweets</i> por semana vs. Novos casos por semana"
)

# Set y-axes titles
fig.update_yaxes(title_text="Novos casos por semana", secondary_y=False)
fig.update_yaxes(title_text="<i>Retweets</i> por semana", secondary_y=True)

fig.write_html('cases_vs_retweets.html')
fig.write_image('svg/cases_vs_retweets.svg', width=1280, height=720)

### Plotando gráfico <i>Retweets</i> por semana vs. Novos casos por semana (MM)

In [121]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=rt_df['count'], x=rt_df['date'], mode='lines', name='<i>Retweets</i> por semana'), secondary_y=True)
fig.add_trace(go.Scatter(y=indicadores['nw_cases_mm'], x=indicadores['date'], mode='lines', name='Novos casos por semana (MM)'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="<i>Retweets</i> por semana vs. Novos casos por semana (MM)"
)

# Set y-axes titles
fig.update_yaxes(title_text="Novos casos por semana (MM)", secondary_y=False)
fig.update_yaxes(title_text="<i>Retweets</i> por semana", secondary_y=True)

fig.write_html('cases_vs_retweets_mm.html')
fig.write_image('svg/cases_vs_retweets_mm.svg', width=1280, height=720)

# Novos casos por semana e Tweets + Retweets

### Plotando gráfico <i>Tweets + Retweets</i> por semana vs. Novos casos por semana

In [122]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=total_df['count'], x=total_df['date'], mode='lines', name='<i>Tweets + Retweets</i> por semana'), secondary_y=True)
fig.add_trace(go.Scatter(y=indicadores['nw_cases'], x=indicadores['date'], mode='lines', name='Novos casos por semana'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="<i>Tweets + Retweets</i> por semana vs. Novos casos por semana"
)

# Set y-axes titles
fig.update_yaxes(title_text="Novos casos por semana", secondary_y=False)
fig.update_yaxes(title_text="<i>Tweets + Retweets</i> por semana", secondary_y=True)

fig.write_html('cases_vs_totaltweets.html')
fig.write_image('svg/cases_vs_totaltweets.svg', width=1280, height=720)

### Plotando gráfico <i>Tweets + Retweets</i> por semana vs. Novos casos por semana (MM)

In [123]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=total_df['count'], x=total_df['date'], mode='lines', name='<i>Tweets + Retweets</i> por semana'), secondary_y=True)
fig.add_trace(go.Scatter(y=indicadores['nw_cases_mm'], x=indicadores['date'], mode='lines', name='Novos casos por semana (MM)'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="<i>Tweets + Retweets</i> por semana vs. Novos casos por semana (MM)"
)

# Set y-axes titles
fig.update_yaxes(title_text="Novos casos por semana (MM)", secondary_y=False)
fig.update_yaxes(title_text="<i>Tweets + Retweets</i> por semana", secondary_y=True)

fig.write_html('cases_vs_totaltweets_mm.html')
fig.write_image('svg/cases_vs_totaltweets_mm.svg', width=1280, height=720)